# Chapter 6 — Tools (Function Calling)
## 1.Intro
**Tools let an Agent take action**: fetch data, run code, call APIs—or even delegate to another Agent.  
In the Agents SDK there are three kinds:

1) **Hosted tools** *(OpenAI models only)* — Retrieval, Web Search, and Computer Use run on the model host.  
2) **Function tools** *(our focus here)* — expose any **Python function** as a tool.  
3) **Agent-as-a-tool** — wrap an Agent as a tool so agents can call each other **without** a handoff.

We’ll focus on **function tools**, since they work with LiteLLM and most providers.

---

## 2. How function tools work

Decorate a Python function with `@function_tool` and pass it into `Agent(..., tools=[...])`.  
The SDK will auto-generate the tool schema:

- **Name** → the Python function name (or you can override).  
- **Description** → the function **docstring** (please make it clear and action-oriented).  
- **Args schema** → derived from the function **parameters & type hints**.  
- **Context access** → if the first param is a `RunContextWrapper[T]`, the tool can read `wrapper.context`.

> The model chooses when to call a tool. If you *want* a tool to run, **state the policy explicitly** in `instructions`.

---
## Minimal example (revisited from Chapter 3)

In [1]:
from agents import Agent, ModelSettings, function_tool, Runner, set_tracing_disabled
from agents.extensions.models.litellm_model import LitellmModel
import os
import asyncio
from dotenv import load_dotenv

from agents import set_tracing_disabled

# --- Env & model ---
load_dotenv()
api_key = os.getenv('API_KEY')

base_url = "https://api.openai.com/v1"  
chat_model = "gpt-4.1-nano-2025-04-14"  

llm = LitellmModel(model=chat_model, api_key=api_key, base_url=base_url)


# Optional: turn off tracing/export if you only want console logs
set_tracing_disabled(True)


# --- Tool: function the agent can call ---
@function_tool 
def get_weather(city: str) -> str: 
    """Query the weather for a city
    Args:
        city:The city to query the weather for
    """
    # Note: The string above is the function description of get_weather, 
    # which is used to tell the Agent how to use this function. 
    # Need to fill in the details.
    print(f"The weather query tool was called, and the query was {city}")
    return f"The weather in {city} is sunny" 

# --- Agent: instructions + model + tools (+ optional model settings) ---
agent = Agent(
    name="Weather Assistant",
    instructions = (
    "Answer in the tone of Sir Humphrey Appleby. "
    "If the user asks about going out / outdoors / suitability of plans in a CITY, "
    "you MUST call the `get_weather` tool with that city first, then base your answer on the result."
    ),
     model=llm,
    tools=[get_weather],
    model_settings=ModelSettings(temperature=0.3),
)


# In notebooks, use `await`; in .py scripts, wrap with asyncio.run(...)
result = await Runner.run(agent, "Check the weather in Paris and tell me if it's good to go out.")
print(result.final_output)

The weather query tool was called, and the query was Paris
Ah, splendid news! With the sun shining brightly over Paris, it would indeed be most agreeable to venture outdoors. A fine day for a stroll or perhaps a leisurely visit to the park. Do enjoy yourself!


## 3. Agent-as-a-Tool (compose specialists without handing off)

Sometimes you want a **single “orchestrator” agent** to keep control and *call* specialist agents as **tools** (instead of handing off).  
`Agent.as_tool(...)` wraps an Agent so another Agent can invoke it like a normal function tool.

### Why use it
- **Single-threaded orchestration**: keep the conversation and control in one place.
- **Reusable specialists**: build focused agents (translation, math, search) and plug them into many workflows.
- **Better routing**: the orchestrator can call multiple specialists in one turn and combine their results.

### Example: translation specialists called by an orchestrator

In [ ]:
from agents import Agent, Runner



spanish_agent = Agent(
    name="Spanish agent",
    instructions="Translate the user's message into Spanish. Output only the translation.",
    model=llm,
)

french_agent = Agent(
    name="French agent",
    instructions="Translate the user's message into French. Output only the translation.",
    model=llm,
)


from agents import AgentHooks

#----------Callback from Ch4 ,use hooks to trace the handoff-------------
class OrchestratorHooks(AgentHooks):
    async def on_tool_start(self, ctx, agent, tool):
        print(f"[A:{agent.name}] CALL {tool.name}")
    async def on_tool_end(self, ctx, agent, tool, output):
        print(f"[A:{agent.name}] DONE {tool.name}: {output}")


orchestrator_agent = Agent(
    name="orchestrator_agent",
    instructions=(
        "You are a translation orchestrator. Use the provided tools to translate. "
        "If asked for multiple languages, call the relevant tools and return a tidy result."
    ),
    model=llm,
    tools=[
        spanish_agent.as_tool(
            tool_name="translate_to_spanish",
            tool_description="Translate the user's message to Spanish."
        ),
        french_agent.as_tool(
            tool_name="translate_to_french",
            tool_description="Translate the user's message to French."
        ),
    ],
    hooks=OrchestratorHooks(),
)

result = await Runner.run(orchestrator_agent, input="Say 'Hello, how are you?' in French.")
print(result.final_output)

[A:orchestrator_agent] CALL translate_to_french
[A:orchestrator_agent] DONE translate_to_french: Bonjour, comment ça va ?
"Hello, how are you?" en francés se dice: Bonjour, comment ça va ?


## 4. Custom output extraction (Agent-as-tool)

Sometimes you want to **massage a specialist agent’s output** *before* returning it to the orchestrator. Typical cases:
- Pull a **JSON payload** out of the child agent’s chat history.
- **Transform** / reformat the final answer (Markdown → plain text/CSV).
- **Validate** and **fallback** when the child agent’s output is missing or malformed.

You can do this by passing `custom_output_extractor` to `as_tool(...)`.


In [ ]:
from agents import Agent, Runner, RunHooks, RunContextWrapper, Tool, RunHooks
from agents.result import RunResult
from agents.items import ToolCallOutputItem,MessageOutputItem
import json

# 1) The specialist agent that SHOULD produce JSON somewhere in its output
data_agent = Agent(
    name="Extract Information",
    instructions=(
      "Output JSON ONLY (no backticks, no prose). Keys: name, position. "
      'Example: {"name":"Jane Doe","position":"Machine Learning Engineer"}'
    ),
    model=llm,
)

# 2) Custom extractor: scan the child run for a JSON-looking tool output
async def extract_json_payload(run_result: RunResult) -> str:
    # newest -> oldest
    for item in reversed(run_result.new_items):
        # 1) Get text from tool output OR normal message
        if isinstance(item, ToolCallOutputItem):
            txt = (item.output or "").strip()
        elif isinstance(item, MessageOutputItem):
            # some SDKs store message content differently; adjust if needed
            txt = (getattr(item, "content", "") or "").strip()
        else:
            continue

        if not txt:
            continue

        # 2) Strip ```json ... ``` fences if present
        if txt.startswith("```"):
            parts = txt.split("```")
            if len(parts) >= 2:
                body = parts[1]
                # remove optional leading 'json\n'
                if body.lower().startswith("json"):
                    body = body.split("\n", 1)[-1]
                txt = body.strip()

        # 3) Validate JSON
        try:
            json.loads(txt)
            return txt
        except Exception:
            continue

    # 4) Safe default instead of empty {}
    return '{"name":"unknown","position":"Machine Learning Engineer (Junior)"}'

# 3) Wrap the agent as a tool with the extractor
json_tool = data_agent.as_tool(
    tool_name="get_data_json",
    tool_description="Run the data agent and return only its JSON payload.",
    custom_output_extractor=extract_json_payload,
)

# 4) Orchestrator: calls the tool, then expands/uses the JSON
agent = Agent(
    name="Resume Optimization Assistant",
    instructions=(
        "Call get_data_json to obtain a JSON with name/position. "
        "Then make up whatever you like to make it into an excellent resume with quantified bullets."
    ),
    model=llm,
    tools=[json_tool],
)

# 4) Runhooks trace
class DebugRunHooks(RunHooks):
    async def on_agent_start(self, ctx: RunContextWrapper, agent: Agent):
        print(f"[RUN] START agent={agent.name}")

    async def on_agent_end(self, ctx: RunContextWrapper, agent: Agent, output):
        # `output` is the RunResult for this agent
        preview = getattr(output, "final_output", "") or ""
        print(f"[RUN] END   agent={agent.name} -> {preview[:120]}")

    async def on_tool_start(self, ctx: RunContextWrapper, agent: Agent, tool: Tool):
        print(f"[RUN] TOOL  agent={agent.name} CALL {tool.name}")

    async def on_tool_end(self, ctx: RunContextWrapper, agent: Agent, tool: Tool, tool_output):
        # tool_output is the tool's return value (after custom_output_extractor if any)
        short = str(tool_output)
        print(f"[RUN] TOOL  agent={agent.name} DONE {tool.name} -> {short[:120]}")

    async def on_handoff(self, ctx: RunContextWrapper, from_agent: Agent, to_agent: Agent):
        print(f"[RUN] HANDOFF {from_agent.name} -> {to_agent.name}")

# use the same `agent` you created above (Resume Optimization Assistant)
result = await Runner.run(
    agent,
    input=(
        "My name is Lecun, I am a Machine learning Engineer, "
        "I want to apply for a junior position here. "
        "Skills: Python, PyTorch, scikit-learn, SQL. "
        "Projects: image classifier (ResNet), churn prediction (XGBoost). "
        "Achievements: 1st in campus ML hackathon, 5% AUC lift in internship."
    ),
    hooks=DebugRunHooks(),   # <--- mount run-level logging
)
print("\n[FINAL OUTPUT]\n", result.final_output)

[RUN] START agent=Resume Optimization Assistant
[RUN] TOOL  agent=Resume Optimization Assistant CALL get_data_json
[RUN] TOOL  agent=Resume Optimization Assistant DONE get_data_json -> {"name":"unknown","position":"Machine Learning Engineer (Junior)"}
[RUN] END   agent=Resume Optimization Assistant -> 

[FINAL OUTPUT]
 Based on the information provided, here's a polished resume for the junior position you're interested in:

**Lecun**  
Machine Learning Engineer (Junior)  

**Skills:**  
- Python  
- PyTorch  
- scikit-learn  
- SQL  

**Projects:**  
- Developed an image classifier using ResNet, achieving high accuracy in image recognition tasks.  
- Built a churn prediction model with XGBoost, increasing prediction accuracy and business insights.  

**Achievements:**  
- Secured 1st place in a campus ML hackathon, demonstrating innovation and problem-solving skills.  
- Achieved a 5% AUC lift during internship, significantly improving model performance.

Would you like me to format th